# Four panel model

Introduction:
------------------

Four-Panel Model (Four panels per grid cell, 4 orientations): In this model,for each time-step the power output is calculated from four tilted PV panels per grid cell. They are oriented towards North, East, South and West with a user-selectable and optimizable slope. The model separately considers flat and pitched roofs. For the flat roofs, PV panels are considered South facing for power output estimation with optimised slope. For pitched roofs, the summed PV power output of the four panels (equally oriented in four cardinal directions) is used. The PV power output from flat and pitched roofs are scaled using the respective projected roof area and totalled over the grid cell to calculate a total idealized PV power output. Finally the output is reduced due to effects of geometric placement and shading.

Methodology:
------------------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms and later the relationships were evaluated using 15 evaluation areas.

Parametrized parameters, namely placement efficiency and shading efficiency are generated from the training areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Shading efficiency calculates the PV power output with shading losses instead of using the SEBE model. 

Significance:
------------------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.


Assumptions/Limitations:
------------------------------------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.
2. Placement efficiency loss was calculated based on a standard PV panel having dimensions (1m x 1.6m)
3. PV panels are tilted to an optimal angle of 40 deg and on flat roofs two rows of PV arrays are seperated by 1.5 m

Output:
-----------

Hourly PV power output [MW km-2] for an urban area

In [75]:
    """

    Input Parameters
    ----------
        A_G          : Total plan area of grid cell [𝑚2]
        A_R          : Projected plan area of all rooftop segments in a grid cell [𝑚2]
        λ_p          : Projected plan area fraction of building segments (A_R/A_G)
        l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                       which tells you how far the shadows get on the roof and is calculated 
                       by taking the square root of the average segment area.
        f_f          : Fraction of projected rooftop segment areas that are flat (0-1)
        f_p          : Scaling fator coefficient(f_p = 1 if all the rooftop segments are covered with panels)
        α_s          : System albedo for an urban area(0-1)
        
        l            : Length of PV panel in portait mode [m]
        w            : Width of PV panel in portait mode [m]
        η_p          : Efficiency of PV panel [%]
        α_T          : Temperature coefficient of PV panel [1/K]
        η_W          : Wiring losses
        d            : Panel row spacing, can be optimized according to technical & Economical parameters
                       In this script, d = 1.5 m
        β_o          : Optimal slope of PV panels on flat rooftops, can be optimized according to location
                       In this script, β_0 = 40° for the city of Berlin.
          
        metadata     : Input meteorological data specifically formatted using 
                       UMEP -> Pre-processing -> Meteorological data -> Prepare existing data
                       Here, a sample Random Meteorological Day(RMD) for the month of June has been given as an example.
                       Annual weather data for the city of Berlin has been attached for reference.
        sol_angles   : Sun position angles for the urban area.
                       Here, solar angles for the RMD - June has been given as example 
        
        Note: 1. In this script, l, w, pv_eff, alpha values has been taken for a standard PV module 
                 but the user can change these parameters according to their module.
              2. lamda_p, l_s, f_f and urban_area values has been taken for an random urban area in the city of Berlin
              3. Input files and parameters can be found under the folder "Datasets" in Github

    """

import numpy as np
import pandas as pd
import math
import pvlib 

# Urban form parameters
A_G         = 265200
λ_p         = 0.398  
l_s         = 9.36  
f_f         = 0.413  
f_p         = 1
α_s         = 0.2

# PV panel characteristics
l           = 1
w           = 1.6 
η_p         = 0.177 
α_T         = -0.0039
η_W         = 0.8
d           = 1.5
β_o         = 40

# Meteorological data
metdata     = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24)) # Input file is located under Datasets folder
sol_angles  = pd.read_csv(f'june_angles_RMD.csv',index_col=0) # Input file is located under Datasets folder

In [76]:
  """
    Constants
    ---------
        T_r          : Reference PV cell temperature, default value = 25 °C
        k_t          : Reduction factor, default value = 0.05 °C/(W/𝑚2)
            
            Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
            differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
            A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
            (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
            building integrated systems.
            
            References:
            ----------
            
            Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
            Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
            and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
            52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002
            
            King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.
            
            
  """

T_r      = 25
k_t      = 0.05

In [77]:
  """
    Theoretical PV panel area
    ------------------------
    
        A_P     : PV panel area [𝑚2]
        A_Pt    : Area of PV panel when it is tilted to an angle [𝑚2]
        A_Ptd   : Area of PV panel when it is tilted and separated by a distance d [𝑚2]
        A_T_4P  : PV panel area is calculated using rooftop building segment area without considering the geometry, tilt, 
                  and aspect of the building segment. [m2]
        
  """
A_P      = l * w
A_Pt     = l * (math.cos(math.radians(β_o))) * w
A_Ptd    = l * (math.cos(math.radians(β_o)) * w + d)
A_R      = λ_p * A_G

A_T_4P = ((f_f / A_Ptd) + ((1 - f_f) / A_Pt)) * A_P * A_R


# theoretical_PVP = (f_f * lamda_p / ((panel_area * math.cos(math.radians(β_o))) + d)) + ((1 - f_f) * lamda_p / (panel_area * math.cos(math.radians(β_o))))

In [78]:
  """
    Parametrized Equations
    ----------------------
    
        ϵ_p          : Placement efficiency is the ratio of geometrically installed PV panels
                       to the theoretical PV panels.
                
        ϵ_s_dir      : Direct shading efficiency loss factor is the reduction of direct POA irradiance due to shading 
                       and is estimated by comparing the direct POA irradiance for Model 4P with building resolving model
                       
        ϵ_s_dif      : Diffuse shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the diffuse POA irradiance for Model 4P with building resolving model
        
        ϵ_s_ref      : Reflected shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the reflected POA irradiance for Model 4P with building resolving model
                
  """
    
ϵ_p     = (-0.0063 * l_s + 0.354)
ϵ_s_dir = (-0.00034 * l_s - 0.00391)
ϵ_s_dif = (-0.00637 * l_s + 0.36056)
ϵ_s_ref = (0.00799 * l_s + 0.60976)

In [79]:
  """
    Geometric PV panel area
    -----------------------
        
        A_3_4P: Area of PV panels that could be in a grid cell considering parametrized placement efficiency loss [m2]

  """
A_3_4P = A_T_4P * (1 - ϵ_p)

# Dividing the panel area into flat and pitched and panel area on pitched roofs are equally divided into four orientations.
# 0 = North, 90 - East, 180 - South, 270 - West 

panel_area = {
180     :   A_3_4P * f_f, # panels over flat rooftop segments
0       :   A_3_4P * (1 - f_f) * 0.25,
90      :   A_3_4P * (1 - f_f) * 0.25,
180.01  :   A_3_4P * (1 - f_f) * 0.25, # 180.01 is taken for differentiating from flat panels @ 180
270     :   A_3_4P * (1 - f_f) * 0.25
}

In [82]:
pv_power = pd.DataFrame()
temp     = pd.DataFrame()

def four_panel(slope, p_area, aspect):
    
    """
    Function
    --------
    
        For calculating PV power output using direct, diffuse and reflected irradiance and parametrized equations

    """

    for i in range(metdata.shape[0]):

        if metdata[[i], 14] > 0: # Considered only the hours when Global horizontal irradiance > 0 W/m2

            Glob_hor = metdata[[i], 14]
            Diff_hor = metdata[[i], 21]
            Dir_hor  = metdata[[i], 22]
            phi      = sol_angles.loc[int(metdata[[i], 2]), 'Azimuth']
            γ        = sol_angles.loc[int(metdata[[i], 2]), 'Solar_Elevation']
            zenith   = sol_angles.loc[int(metdata[[i], 2]), 'Zenith']
            aoi      = pvlib.irradiance.aoi(slope, aspect, zenith, phi) # Angle of Incidence calculated using pvlib (python library)
            metdata1 = metdata[[i], :]

            if γ > 0 : # Considered only Sunshine hours (Solar elevation > 0)

                lamda = math.cos(math.radians(phi - aspect)) + (math.tan(math.radians(γ)) / math.tan(math.radians(slope)))

                if lamda <= 0 :

                    f_s1 = 1
                    f_s2 = 1

                else:

                    f_s1 = 0
                    x_l  = (math.cos(math.radians(slope)) + d/l) * math.sin(math.radians(phi - aspect)) / lamda
                    y_l  = 1 - ((math.cos(math.radians(slope)) + d/l) * math.tan(math.radians(γ)) / (lamda * math.sin(math.radians(slope))))

                    if y_l <=0:

                        f_s2 = 0

                    elif abs(x_l) > (w / l):

                        f_s2 = 0

                    else:

                        f_s2 = (1 - (abs(x_l)/ (w / l))) * y_l

                f_sky1 = (1 + math.cos(math.radians(slope))) / 2

                B       = Dir_hor * (math.cos(math.radians(aoi)) / math.sin(math.radians(γ)))                
                poa_dir = (1 - f_s1) * B * (1 - (ϵ_s_dir * γ + 1))
                poa_dif = f_sky1 * Diff_hor * (1 - ϵ_s_dif)
                ref_dif = α_s * Glob_hor * (1 - f_sky1) * (1 - ϵ_s_ref)
                
                pv_power.loc[i,f'Year']      = int(metdata1[0,0])
                pv_power.loc[i,f'Day']       = int(metdata1[0,1])
                pv_power.loc[i,f'Hour']      = int(metdata1[0,2])
                temp.loc[i,f'Hour']          = int(metdata1[0,2])
                temp.loc[i,f'dir_{aspect}']  = poa_dir * η_p * (1 + α_T * ((metdata[[i], 11] + k_t * poa_dir) - T_r)) * p_area * η_W * f_p 
                temp.loc[i,f'dif_{aspect}']  = poa_dif * η_p * (1 + α_T * ((metdata[[i], 11] + k_t * poa_dif) - T_r)) * p_area * η_W * f_p 
                temp.loc[i,f'ref_{aspect}']  = ref_dif * η_p * (1 + α_T * ((metdata[[i], 11] + k_t * ref_dif) - T_r)) * p_area * η_W * f_p 


for aspect, p_area in panel_area.items():
        
    four_panel(β_o, p_area, aspect)

pv_power['direct [MW km-2]']    = (temp['dir_180'] + temp['dir_0'] + temp['dir_90']+ temp['dir_180.01']+ temp['dir_270']) / A_G 
pv_power['diffuse [MW km-2]']   = (temp['dif_180'] + temp['dif_0'] + temp['dif_90']+ temp['dif_180.01']+ temp['dif_270']) / A_G 
pv_power['reflected [MW km-2]'] = (temp['ref_180'] + temp['ref_0'] + temp['ref_90']+ temp['ref_180.01']+ temp['ref_270']) / A_G 

pv_power['Total_PVP [MW km-2]'] = (pv_power['direct [MW km-2]'] + pv_power['diffuse [MW km-2]'] + pv_power['reflected [MW km-2]']) 

In [83]:
"""
    Output
    -------
        Hourly PV power output [MW km-2] for an urban area
        
"""
pv_power

,Year,Day,Hour,direct [MW km-2],diffuse [MW km-2],reflected [MW km-2],Total_PVP [MW km-2]
5,2019.0,167.0,5.0,0.057906,0.666268,0.012599,0.736773
6,2019.0,171.0,6.0,0.325005,1.427906,0.043111,1.796022
7,2019.0,160.0,7.0,2.080042,2.405755,0.086606,4.572403
8,2019.0,152.0,8.0,2.114862,3.313178,0.097579,5.525618
9,2019.0,175.0,9.0,5.151254,2.840767,0.172368,8.164388
10,2019.0,179.0,10.0,0.819075,6.334633,0.095668,7.249376
11,2019.0,169.0,11.0,8.507923,3.458890,0.224533,12.191346
12,2019.0,174.0,12.0,9.527293,3.486676,0.240416,13.254385
13,2019.0,156.0,13.0,8.064322,4.331883,0.217524,12.613730
14,2019.0,165.0,14.0,9.027303,3.337463,0.226809,12.591575
